In [1]:
# 导入所需要的库
from langchain.llms import OpenAI
from langchain.chains import LLMChain, SequentialChain
from langchain.prompts import PromptTemplate

# 第一个LLMChain：生成鲜花的介绍
llm = OpenAI(temperature=.7)
template = """
你是一个植物学家。给定花的名称和类型，你需要为这种花写一个200字左右的介绍。
花名: {name}
颜色: {color}
植物学家: 这是关于上述花的介绍:"""
prompt_template = PromptTemplate(
    input_variables=["name", "color"],
    template=template
)
introduction_chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    output_key="introduction" # 这里的输出，传到下一个链中
)

/Users/lianggao/miniconda3/envs/langchain-demo/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
/Users/lianggao/miniconda3/envs/langchain-demo/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [2]:
# 第二个LLMChain：根据鲜花的介绍写出鲜花的评论
template = """
你是一位鲜花评论家。给定一种花的介绍，你需要为这种花写一篇200字左右的评论。
鲜花介绍:
{introduction}
花评人对上述花的评论:"""
prompt_template = PromptTemplate(
    input_variables=["introduction"], # 接收上一个链的输出
    template=template
)
review_chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    output_key="review"
)

In [4]:
# 第三个LLMChain：根据鲜花的介绍和评论写出一篇自媒体的文案
template = """
你是一家花店的社交媒体经理。给定一种花的介绍和评论，你需要为这种花写一篇社交媒体的帖子，300字左右。
鲜花介绍:
{introduction}
花评人对上述花的评论:
{review}
社交媒体帖子:
"""
prompt_template = PromptTemplate(
    input_variables=["introduction", "review"], # 接收上两个链的输出
    template=template 
)
social_post_chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    output_key="social_post_text" # 输出字段
)



In [5]:
# 总的链：按顺序运行三个链
overall_chain = SequentialChain(
    chains=[introduction_chain, review_chain, social_post_chain],
    input_variables=["name", "color"],
    output_variables=["introduction", "review", "social_post_text"], # 所有链的输出
    verbose=True
)

# 运行链并打印结果
result = overall_chain({
    "name": "玫瑰",
    "color": "黑色"
})
print(result)


/Users/lianggao/miniconda3/envs/langchain-demo/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...

> Finished chain.
{'name': '玫瑰', 'color': '黑色', 'introduction': '\n\n玫瑰是一种美丽的花朵，也是最受欢迎的花之一。它的名字来自于拉丁文“rosa”，意为“红色”，但实际上，它有许多不同的颜色，包括黑色。黑色玫瑰是一种罕见的花朵，它给人一种神秘和优雅的感觉。\n\n黑色玫瑰的花瓣是深黑色的，有时候带有紫色的色调。它们通常比普通的玫瑰花更大，花瓣也更加饱满。花朵的形状也有所不同，有些是像球状的，有些则是像杯子一样。不管是哪种形状，黑色玫瑰都令人惊叹。\n\n这种花的起源并不明', 'review': '\n\n黑色玫瑰是一种充满神秘感的花朵，它的深黑色花瓣散发着一种优雅的气息。这种罕见的花朵给人一种神秘而又迷人的感觉，让人不禁沉醉在它的美丽中。\n\n一朵黑色玫瑰的花瓣比普通的玫瑰更加饱满，花朵的大小也更加惊人。有的花瓣像球状，有的像杯子一样，但不管是哪种形状，都散发出一种独特的魅力。\n\n这种花的起源仍然是一个谜，但它的存在却让人惊叹不已。它的名字虽然是“黑色玫瑰”，但实际上它的', 'social_post_text': '\n大家好！今天我要向大家介绍一种美丽的花朵——黑色玫瑰。它的名字来自于拉丁文“rosa”，意为“红色”，但实际上，它有多种颜色，其中最特别的就是深黑色。这种罕见的花朵给人一种神秘和优雅的感觉。\n\n黑色玫瑰的花瓣是深黑色的，有时候带有紫色的色调。它们比普通的玫瑰花更大，花瓣也更加饱满。花朵的形状也有所不同，有些是像球状的，有些则是像杯子一样。无论哪种形状，黑色玫瑰都令人惊叹。\n\n这种花的起源仍然是一个谜，但它的存在'}
